In [1]:
import xarray as xr

In [2]:
infile = '/g/data/xv83/dbi599/precip_c5-d60-pX-f6_19861101-19871101_atmos_isobaric_daily.zarr.zip'

In [5]:
ds = xr.open_zarr(infile)

In [6]:
ds

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lat: 19, lead_time: 3653, lon: 19)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1986-11-01 1987-11-01
  * lat        (lat) float64 -45.51 -43.48 -41.46 ... -13.15 -11.12 -9.101
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3647 3648 3649 3650 3651 3652
  * lon        (lon) float64 111.2 113.8 116.2 118.8 ... 148.8 151.2 153.8 156.2
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(3653, 2), meta=np.ndarray>
Data variables:
    precip     (init_date, lead_time, ensemble, lat, lon) float32 dask.array<chunksize=(1, 28, 96, 19, 19), meta=np.ndarray>

In [ ]:
#f6_precip_ann = lf.reindex_forecast(f6_precip).resample(time="A-DEC", label='right').apply(lf.sum_min_samples, dim='time', min_samples=365).rename('precip_ann')

In [43]:
def reindex_forecast(ds):
    """Restack by time a forecast dataset stacked by lead_time"""

    to_concat = []
    for init_date in ds['init_date']:
        fcst = ds.sel({'init_date': init_date})
        to_concat.append(fcst.swap_dims({'lead_time': 'time'}))
    concat = xr.concat(to_concat, dim='init_date')

    return concat

In [44]:
ds = reindex_forecast(ds)

In [45]:
ds

,Array,Chunk
Bytes,1.11 GB,3.88 MB
Shape,"(2, 4018, 96, 19, 19)","(1, 28, 96, 19, 19)"
Count,4427 Tasks,522 Chunks
Type,float32,numpy.ndarray


In [46]:
ds.resample(time="A-DEC", label='right')

DatasetResample, grouped over '__resample_dim__' 
12 groups with labels 1986-12-31, ..., 1997-12-31.